In [222]:
import numpy as np
import matplotlib.pyplot as plt

from termcolor import colored, cprint

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data.dataset import Dataset
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [223]:
Data = np.loadtxt('dataWhole.txt', dtype=str, delimiter = '\n')


In [224]:
print(Data.shape)
separate = int(data.shape[0] * 0.8)

(1941123,)


In [225]:
dataTrain = Data[0:separate]
dataValid = Data[separate:]

In [226]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [227]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

In [228]:
def toBoards(game, color, turn):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, turn + 1):
        turnC = toTurn(game[i])
        if curPlayer == 1:
            blackBoard[turnC] = 1
        else:
            whiteBoard[turnC] = -1
        curPlayer *= -1
    curBoard = np.zeros(shape=(3, 15, 15))
    curBoard[0,:] = playerBoard
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    label = toTurn(game[turn + 1])
    return curBoard, label

In [229]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [230]:
def takeRandom(data):
    game = []
    while len(game) <= 4:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = toBoards(game, color, turn * 2)
    else:
        board, label = toBoards(game, color, turn * 2 + 1)

    return board, label[0] * 15 + label[1]

In [231]:
def makeBatch(size, data):
    batch = np.zeros(shape=(size, 3, 15, 15))
    labels = np.zeros(shape=size, dtype=np.long)
    for i in range(size):
        batch[i,:,:,:], labels[i] = takeRandom(data)
    return torch.from_numpy(batch), torch.from_numpy(labels)

In [251]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(3, affine=False)
        self.pad2 = nn.ConstantPad2d(2, 0)
        self.pad1 = nn.ConstantPad2d(1, 0)

        self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv2 = nn.Conv2d(64, 32, 3)
        self.conv3 = nn.Conv2d(32, 16, 3)
        self.conv4 = nn.Conv2d(16, 1, 5)

    def forward(self, x):
        x = self.batchnorm(x.float())
        x = self.pad1(F.relu(self.conv1(x)))
        x = self.pad1(F.relu(self.conv2(x)))
        x = self.pad1(F.relu(self.conv3(x)))
        x = self.pad2(F.relu(self.conv4(x)))

        x = x.view(-1, 225)
 
        return x

In [252]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
batchNum = 1000000
epochNum = 2
batchSize = 32
trainPart = 0.8
validPart = 0.2
optimizer = optim.Adam(net.parameters(), lr=0.0001)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [253]:
for epoch in range(epochNum):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(batchNum):
        # get the inputs
        inputs, labels = makeBatch(batchSize, dataTrain)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 4.827
[1,   400] loss: 4.198
[1,   600] loss: 3.998
[1,   800] loss: 3.871
[1,  1000] loss: 3.796
[1,  1200] loss: 3.713
[1,  1400] loss: 3.655
[1,  1600] loss: 3.602
[1,  1800] loss: 3.549
[1,  2000] loss: 3.524
[1,  2200] loss: 3.488
[1,  2400] loss: 3.421
[1,  2600] loss: 3.376
[1,  2800] loss: 3.377
[1,  3000] loss: 3.323
[1,  3200] loss: 3.300
[1,  3400] loss: 3.290
[1,  3600] loss: 3.277
[1,  3800] loss: 3.245
[1,  4000] loss: 3.217
[1,  4200] loss: 3.184
[1,  4400] loss: 3.193
[1,  4600] loss: 3.142
[1,  4800] loss: 3.176
[1,  5000] loss: 3.152
[1,  5200] loss: 3.114
[1,  5400] loss: 3.124
[1,  5600] loss: 3.153
[1,  5800] loss: 3.075
[1,  6000] loss: 3.125
[1,  6200] loss: 3.078
[1,  6400] loss: 3.083
[1,  6600] loss: 3.063
[1,  6800] loss: 3.035
[1,  7000] loss: 3.047
[1,  7200] loss: 3.086
[1,  7400] loss: 3.031
[1,  7600] loss: 3.084
[1,  7800] loss: 3.034
[1,  8000] loss: 2.997
[1,  8200] loss: 2.999
[1,  8400] loss: 2.995
[1,  8600] loss: 2.994
[1,  8800] 

[1, 71600] loss: 2.536
[1, 71800] loss: 2.568
[1, 72000] loss: 2.567
[1, 72200] loss: 2.538
[1, 72400] loss: 2.566
[1, 72600] loss: 2.566
[1, 72800] loss: 2.520
[1, 73000] loss: 2.567
[1, 73200] loss: 2.586
[1, 73400] loss: 2.517
[1, 73600] loss: 2.586
[1, 73800] loss: 2.562
[1, 74000] loss: 2.538
[1, 74200] loss: 2.581
[1, 74400] loss: 2.564
[1, 74600] loss: 2.539
[1, 74800] loss: 2.541
[1, 75000] loss: 2.563
[1, 75200] loss: 2.549
[1, 75400] loss: 2.511
[1, 75600] loss: 2.600
[1, 75800] loss: 2.542
[1, 76000] loss: 2.561
[1, 76200] loss: 2.528
[1, 76400] loss: 2.523
[1, 76600] loss: 2.536
[1, 76800] loss: 2.543
[1, 77000] loss: 2.555
[1, 77200] loss: 2.555
[1, 77400] loss: 2.520
[1, 77600] loss: 2.571
[1, 77800] loss: 2.549
[1, 78000] loss: 2.533
[1, 78200] loss: 2.598
[1, 78400] loss: 2.530
[1, 78600] loss: 2.518
[1, 78800] loss: 2.495
[1, 79000] loss: 2.535
[1, 79200] loss: 2.482
[1, 79400] loss: 2.547
[1, 79600] loss: 2.547
[1, 79800] loss: 2.525
[1, 80000] loss: 2.564
[1, 80200] 

KeyboardInterrupt: 

## Validation

In [246]:
correct = 0
total = 0
for i in range(100000):
    if i % 1000 == 999:
        print('Accuracy of the network on the 1000 test images: %5f %%' % (100 * correct / total))    
    inputs, labels = makeBatch(batchSize, dataTrain)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 100000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 1000 test images: 29.886136 %
Accuracy of the network on the 1000 test images: 29.864932 %
Accuracy of the network on the 1000 test images: 29.792222 %
Accuracy of the network on the 1000 test images: 29.773068 %
Accuracy of the network on the 1000 test images: 29.732822 %
Accuracy of the network on the 1000 test images: 29.696095 %
Accuracy of the network on the 1000 test images: 29.629679 %
Accuracy of the network on the 1000 test images: 29.619327 %
Accuracy of the network on the 1000 test images: 29.641141 %
Accuracy of the network on the 1000 test images: 29.623900 %
Accuracy of the network on the 1000 test images: 29.625989 %


KeyboardInterrupt: 

## Game

In [247]:
def toTripleBoard(board, side):
    curBoard = np.zeros(shape=(3, 15, 15), dtype=np.float)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.float)
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.float)

    if side == 1:
        blackTurnBoard = np.ones(shape=(15, 15), dtype=np.float)
    else:
        blackTurnBoard = -np.ones(shape=(15, 15), dtype=np.float)
    curBoard[0,:] = blackTurnBoard
    for i in range(15):
        for j in range(15):
            if board[i, j] == 1:
                blackBoard[i, j] = 1
            if board[i, j] == -1:
                whiteBoard[i, j] = -1
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    return curBoard


In [248]:
def printBoard(curBoard):
    print("----------------------------------------------------------------")
    cprint("     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 ", "blue")
    for i in range(15):
        if i < 10:
            cprint(" " + str(i), "blue", end=" ")
        else:
            cprint(i, "blue", end=" ")
        for j in range(15):
            if curBoard[i, j] == -1:
                print("|", end="")
                cprint(str(-1), 'red', "on_green", attrs=['blink'], end=" ")
            else:
                print("|", end = " ")
                if curBoard[i, j] == 1:
                    cprint(int(curBoard[i, j]), 'yellow', "on_blue", attrs=['blink'], end=" ")
                else:
                    print(int(curBoard[i, j]), end=" ")
        print("|")
        print("----------------------------------------------------------------")
    print()
    return


 ## (we are white)

In [249]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)
    print(netTurn)
    curBoard[netTurn // 15, netTurn % 15] = 1
    printBoard(curBoard)
    
    x, y = list(map(int, input().split()))
    curBoard[x, y] = -1
    #printBoard(curBoard)


tensor([160])
----------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-----------

9 8
tensor([175])
----------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------

12 10
tensor([127])
----------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-----

KeyboardInterrupt: 

## (we are black)

In [250]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    x, y = list(map(int, input().split()))
    curBoard[x, y] = 1
    #printBoard(curBoard)
    
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)

    curBoard[netTurn // 15, netTurn % 15] = -1
    printBoard(curBoard)


6 6
----------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |-1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
---------------------

KeyboardInterrupt: 

In [152]:

text = colored('Hello, World!', 'red', attrs=['reverse', 'blink'])
print(text)
cprint('Hello, World!', 'green', 'on_yellow', attrs=['blink'])

Hello, World!
Hello, World!
